In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('../products/production_metadata/metadata_full.csv')

/tmp/ipykernel_15610/3477465231.py:1: DtypeWarning: Columns (2,5,6,7,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../products/production_metadata/metadata_full.csv')


In [3]:
dfe = pd.read_csv('../products/raw/embeddings.csv', header=None)

In [4]:
dfe = dfe[dfe.index.isin(df.index)]

In [5]:
del df

In [6]:
dfe.reset_index(inplace=True)

In [7]:
dfe.drop_duplicates(subset=[0], inplace=True)

In [9]:
dfe.columns

Index(['index',       0,       1,       2,       3,       4,       5,       6,
             7,       8,
       ...
           759,     760,     761,     762,     763,     764,     765,     766,
           767,     768],
      dtype='object', length=770)

In [10]:
dfe.drop(columns=['index'],inplace=True)

In [14]:
from joblib import parallel_backend

with parallel_backend('loky', n_jobs=4):
    scaler = StandardScaler()
    dfe[dfe.columns[1:]] = scaler.fit_transform(dfe[dfe.columns[1:]])

KeyboardInterrupt: 

In [ ]:
with parallel_backend('loky'):
    kmeanModel = KMeans(n_clusters=5, init='k-means++', n_init=50,verbose=1)
    kmean_labels = kmeanModel.fit_predict(dfe.drop(0, axis=1))
    print(kmean_labels)

## DASK

In [1]:
from dask.distributed import Client, progress
import dask.dataframe as dd
import dask
import pandas as pd

In [2]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit="15GB")
client

2022-08-07 22:24:14,175 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-avjhm4q9', purging
2022-08-07 22:24:14,176 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-p_0050sy', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 4,Total memory: 27.94 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36475,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 27.94 GiB
Comm: tcp://127.0.0.1:35697,Total threads: 2
Dashboard: http://127.0.0.1:39855/status,Memory: 13.97 GiB
Nanny: tcp://127.0.0.1:43857,


In [3]:
dfe = dd.read_csv('../products/raw/embeddings.csv',header=None)
df = pd.read_csv('../products/production_metadata/metadata_full.csv')

/tmp/ipykernel_27805/3910003281.py:2: DtypeWarning: Columns (2,5,6,7,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../products/production_metadata/metadata_full.csv')


In [4]:
dfe = dfe.reset_index(drop=True)

In [5]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
df.set_index('cord_uid',inplace=True)

In [7]:
dfe = dfe[dfe.iloc[:,0].isin(df.index)]

In [8]:
dfe = dfe.rename(columns={0:'cord_uid'})

In [9]:
dfe = dfe.drop_duplicates(subset=['cord_uid']).persist()

In [10]:
from dask_ml.cluster import KMeans
from dask_ml.preprocessing import StandardScaler

In [14]:
scaler = StandardScaler(copy=False)
scaled_dfe = scaler.fit_transform(dfe[dfe.columns[1:]])

In [15]:
scaled_dfe.head()

,1,2,3,4,5,6,7,8,9,10,...,759,760,761,762,763,764,765,766,767,768
0,-0.317739,-1.036730,-0.057281,0.112621,-0.161881,-0.539089,0.722888,0.646912,0.687878,0.942999,...,-0.147209,0.455986,0.698343,0.315879,-0.604479,-0.537778,-0.637476,1.622758,-0.319883,0.194676
1,3.342620,-1.022605,0.896421,0.275025,2.466357,0.520143,-1.285753,-2.100241,-0.073992,0.505982,...,-0.205114,0.121520,-1.362842,-0.903798,0.446947,1.470588,-1.223217,0.920383,-1.511195,0.292459
2,1.361620,-1.261716,1.334113,1.105418,1.693575,0.059347,-0.516698,0.402758,0.834540,0.215621,...,1.341805,0.907070,-0.661467,2.042139,0.098573,0.289445,-0.769019,1.068631,-0.283232,-0.674269
3,2.221773,0.142096,1.387721,-0.115501,1.102069,1.667158,-1.868952,-1.601719,0.310353,-0.629163,...,0.307653,0.503300,-1.952943,0.281496,0.670976,0.137335,-2.452166,-1.120315,0.342070,1.901230
4,0.473530,-0.801315,1.034529,1.243672,2.741525,-1.562764,0.812232,-0.795136,0.817400,0.336953,...,0.810163,-0.263162,0.476509,0.477988,0.180130,-0.258169,-0.461511,0.941739,1.486730,-1.891890


In [16]:
kmeanModel = KMeans(n_clusters=5,init='k-means||',init_max_iter=10)
kmeanModel.fit(scaled_dfe)

2022-08-07 23:38:15,904 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 9.78 GiB -- Worker memory limit: 13.97 GiB
2022-08-07 23:38:16,002 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 9.85 GiB -- Worker memory limit: 13.97 GiB
2022-08-07 23:38:16,100 - distributed.worker_memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. --

In [ ]:
labels = kmeanModel.predict(scaled_dfe)

In [ ]:
from joblib import dump, load
dump(kmeanModel, './kmeanModel-00.joblib') 

In [ ]:
dfe['label']= labels
dfe_label = dfe[['cord_uid','label']]

In [ ]:
dfe_label.to_csv('./labels-*.csv')